<a href="https://colab.research.google.com/github/omniaghazy/Core_X/blob/main/_Gain_Degradation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install pybullet

In [8]:
import pybullet as p
import time
import pybullet_data
import math
import pandas as pd
import os
from typing import List

print("\n--- Starting Gain Degradation Simulation ---")

# ==========================================================
# ⚙️ الـSetup: Connect والـLoad
# ==========================================================
# (يجب إعادة تشغيل Notebook Cell أو عمل p.disconnect() قبل هذا الـBlock)
physicsClient = p.connect(p.DIRECT)
pybullet_data_path = pybullet_data.getDataPath()
p.setAdditionalSearchPath(pybullet_data_path)
p.setGravity(0, 0, -9.81)
p.setRealTimeSimulation(0)
planeId = p.loadURDF("plane.urdf")
startPos = [0, 0, 0.01]
startOrientation = p.getQuaternionFromEuler([0, 0, 0])
robotId = p.loadURDF("kuka_iiwa/model.urdf", startPos, startOrientation, useFixedBase=True)

# إعداد المفاصل والـSensors
numJoints = p.getNumJoints(robotId)
controllable_joints = [i for i in range(numJoints) if p.getJointInfo(robotId, i)[2] == p.JOINT_REVOLUTE]
end_effector_link_index = 6
joint_to_fail = 4
for joint in controllable_joints:
    p.setJointMotorControl2(robotId, joint, p.VELOCITY_CONTROL, force=0) # لا نحتاج Sensors هنا

# --- 5. إعداد حقن العطل (Gain Degradation) ---
INITIAL_P_GAIN = 0.5
MIN_P_GAIN = 0.05
total_steps = 20000
fault_injection_step = 10000
GAIN_DECREMENT = (INITIAL_P_GAIN - MIN_P_GAIN) / (total_steps - fault_injection_step)

data_log: List[List] = []
print(f"✅ Setup Done. Simulation starting for {total_steps} steps.")

# --- 6. اللوب الرئيسي للمحاكاة (توليد البيانات) ---
current_p_gain = INITIAL_P_GAIN
try:
    for step in range(total_steps):

        # أ. حساب المسار (Trajectory) والـIK
        target_x = 0.6 + 0.2 * math.sin(step * 0.01)
        target_y = 0.0
        target_z = 0.8
        target_position = [target_x, target_y, target_z]
        perfect_commands_list = p.calculateInverseKinematics(robotId, end_effector_link_index, target_position)

        # --- ج. (حقن العطل) ---
        if step >= fault_injection_step:
            if step == fault_injection_step:
                print(f"[STEP {step}] --- FAULT INJECTED! Starting P-Gain Decrease. ---")

            # تقليل قيمة الـGain تدريجياً
            current_p_gain = max(current_p_gain - GAIN_DECREMENT, MIN_P_GAIN)

        log_row = [step]

        # --- د. لوب على كل المفاصل للتحكم والتسجيل ---
        for i in range(len(controllable_joints)):
            joint_index = controllable_joints[i]
            target_pos = perfect_commands_list[i]

            # 2. إرسال الأمر للمحاكي (مع تعديل الـGain للمفصل المعطوب)
            final_p_gain = current_p_gain if joint_index == joint_to_fail else INITIAL_P_GAIN

            p.setJointMotorControl2(bodyIndex=robotId, jointIndex=joint_index, controlMode=p.POSITION_CONTROL,
                                    targetPosition=target_pos, positionGain=final_p_gain, velocityGain=0.5)

            # 3. تسجيل البيانات
            joint_state = p.getJointState(robotId, joint_index)
            actual_position = joint_state[0]

            log_row.append(target_pos) # Command Position
            log_row.append(actual_position) # Actual Position (PM Signal)


        log_row.append(current_p_gain) # Gain Label
        data_log.append(log_row)
        p.stepSimulation()

except Exception as e:
    print(f"\nAn error occurred during simulation: {e}")
finally:
    # --- 7. إنهاء الاتصال وحفظ البيانات ---
    p.disconnect()
    if data_log:
        columns = ['Step']
        for j in controllable_joints:
            columns.extend([f'Cmd_J{j}', f'Act_J{j}'])
        columns.append('PGain_Label')

        df = pd.DataFrame(data_log, columns=columns)
        output_filename = "trajectory_Gain_Degradation_data.csv"
        df.to_csv(output_filename, index=False)
        print(f"\n✅ SUCCESS! Data saved to: {output_filename}")


--- Starting Gain Degradation Simulation ---
✅ Setup Done. Simulation starting for 20000 steps.
[STEP 10000] --- FAULT INJECTED! Starting P-Gain Decrease. ---

✅ SUCCESS! Data saved to: trajectory_Gain_Degradation_data.csv


In [9]:
df = pd.read_csv("trajectory_Gain_Degradation_data.csv")
df

,Step,Cmd_J0,Act_J0,Cmd_J1,Act_J1,Cmd_J2,Act_J2,Cmd_J3,Act_J3,Cmd_J4,Act_J4,Cmd_J5,Act_J5,Cmd_J6,Act_J6,PGain_Label
0,0,-3.914271e-12,0.000000,0.501474,0.000000,-1.241552e-12,0.000000,-0.826850,0.000000,8.906893e-13,0.000000e+00,0.169378,0.000000,0.000000,0.000000,0.500000
1,1,6.248322e-05,-0.000071,0.390694,0.250305,-9.656324e-05,-0.000163,-1.019505,-0.413378,2.344212e-06,3.832736e-08,0.216743,0.084697,-0.000338,-0.000338,0.500000
2,2,-3.444251e-03,0.000997,0.361138,0.515617,6.957870e-03,0.009126,-1.052319,-0.752473,-2.893164e-02,-2.922808e-02,0.354495,0.268437,0.020279,0.020279,0.500000
3,3,-5.982692e-03,-0.001412,0.361870,0.646711,1.479473e-02,0.016576,-1.032342,-0.889680,-6.473316e-02,-6.527535e-02,0.509551,0.443982,0.016860,0.016860,0.500000
4,4,-6.449992e-03,-0.004361,0.356185,0.586311,1.827992e-02,0.018906,-1.034815,-0.993101,-8.459542e-02,-8.485173e-02,0.587103,0.551964,0.024761,0.024761,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,-7.893101e-02,-0.079013,1.689926,1.690367,-6.895917e-02,-0.068900,1.682126,1.684395,-1.817281e-01,-1.817268e-01,-0.016308,-0.016689,-0.097287,-0.097287,0.050180
19996,19996,-7.884903e-02,-0.078934,1.689901,1.690355,-6.898227e-02,-0.068921,1.680206,1.682376,-1.928987e-01,-1.928973e-01,-0.016551,-0.016914,0.111046,0.111046,0.050135
19997,19997,-7.877010e-02,-0.078857,1.689867,1.690344,-6.910357e-02,-0.069042,1.678395,1.680780,-1.817448e-01,-1.817434e-01,-0.015776,-0.016176,-0.097287,-0.097287,0.050090
19998,19998,-7.868636e-02,-0.078774,1.689858,1.690312,-6.913006e-02,-0.069066,1.676427,1.678665,-1.929155e-01,-1.929142e-01,-0.016013,-0.016387,0.111046,0.111046,0.050045
